<a href="https://colab.research.google.com/github/TaviTragus/data-science-fundamentals/blob/main/Unit9/TaviTragU9_i_got_bored_of_the_other_thing_and_wanted_to_explore_somthing_else_but_colab_storge_restrictions_(.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Unit 9 Project: Automatic Speech Recognition
The goal of my project is to create a very rough speech to text model trained on the clean librispeech data from hugging face. This is probably going to end up looking a lot more like me fumbling around with structures and not getting anything to work <3.

##Loading the data##

In [2]:
from datasets import load_dataset
from datasets import Audio
from transformers import AutoProcessor

Huggingface has a very convenient method, I just pick a dataset and tell the library to load it. I am downloading like 10000 audio files though, so this took a long time

In [4]:
librispeech = load_dataset("openslr/librispeech_asr", "clean", split = 'train.100')

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train.100 split:   0%|          | 0/28539 [00:00<?, ? examples/s]

Generating train.360 split:   0%|          | 0/104014 [00:00<?, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

From there I went to see what

In [5]:
print(librispeech)

NameError: name 'librispeech' is not defined

In [ ]:
print(librispeech[train])

In [ ]:
columns_to_keep= ["", ""]
all_columns = librispeech[""].column_names
columns_to_remove = set(all_columns) - set(columns_to_keep)

librispeech = librispeech.remove_columns(columns_to_remove)

In [ ]:
librispeech = librispeech.cast_column("audio", Audio(sampling_rate=8000))  #changing from 16kHz to 8kHz

processor = AutoProcessor.from_pretrained("openai/whisper-tiny.en")      #preprocessing, so audio is in the right format. If i understand this correctly, this will make the data
def prepare_dataset(batch):                                                #more understandable for my model
    audio = batch["audio"]                                                 #this comes from the hugging face website.
    batch = processor(audio["array"], sampling_rate=audio["sampling_rate"], text=batch["text"])
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]
    return batch
librispeech = librispeech.map(prepare_dataset, remove_columns=librispeech["train"].column_names)

max_duration = 30.0                                                        #removing audio clips over 30 seconds
def is_audio_length_in_range(input_length):
    return input_length < max_duration
librispeech = librispeech.filter(is_audio_length_in_range, input_columns=["input_length"])


In [ ]:
import torch
import torchaudio
import torch.nn as nn
import torchaudio.transforms as T

import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt